In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import time
from data_loader import get_cifar100_loaders

In [2]:
def train_model(model, train_loader, test_loader, num_epochs, learning_rate, device, model_name="resnet18"):
    """Hàm huấn luyện và đánh giá model (tương tự CustomCNN)."""
    criterion = nn.CrossEntropyLoss()

    # Sử dụng SGD với momentum thường hiệu quả hơn Adam cho fine-tuning ResNet
    # Có thể cần điều chỉnh learning rate và momentum
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)

    # Thêm Learning Rate Scheduler để giảm LR trong quá trình huấn luyện
    # Ví dụ: Giảm LR đi 10 lần sau mỗi 20 epochs
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

    model.to(device)

    best_accuracy = 0.0
    model_save_path = f'{model_name}_cifar100_best.pth' # Tên file lưu model

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc_train = 100.0 * correct_train / total_train

        # Đánh giá trên tập test
        model.eval()
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total_test += labels.size(0)
                correct_test += (predicted == labels).sum().item()

        epoch_acc_test = 100.0 * correct_test / total_test
        end_time = time.time()
        epoch_duration = end_time - start_time

        print(f'Epoch {epoch+1}/{num_epochs} | '
              f'Train Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc_train:.2f}% | '
              f'Test Acc: {epoch_acc_test:.2f}% | LR: {scheduler.get_last_lr()[0]:.6f} | ' # In learning rate hiện tại
              f'Duration: {epoch_duration:.2f}s')

        # Cập nhật scheduler
        scheduler.step()

        # Lưu model tốt nhất
        if epoch_acc_test > best_accuracy:
            best_accuracy = epoch_acc_test
            try:
                torch.save(model.state_dict(), model_save_path)
                print(f'>>> Best model saved to {model_save_path} with Test Accuracy: {best_accuracy:.2f}%')
            except Exception as e:
                print(f"Lỗi khi lưu model: {e}")


    print('Finished Training')
    print(f'Best Test Accuracy achieved: {best_accuracy:.2f}%')

In [ ]:
# --- Hyperparameters ---
NUM_EPOCHS = 50
BATCH_SIZE = 64
LEARNING_RATE = 0.01
IMG_SIZE = 224          # Resize ảnh lên 224x224 cho ResNet pre-trained
USE_AUGMENTATION = True
DATA_DIR = './data_cifar100'
NUM_WORKERS = 4

# --- Thiết bị ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# --- Tải dữ liệu ---
print(f"Đang tải dữ liệu CIFAR-100 và resize về {IMG_SIZE}x{IMG_SIZE}...")
train_loader, test_loader, num_classes = get_cifar100_loaders(
    batch_size=BATCH_SIZE,
    data_dir=DATA_DIR,
    img_size=IMG_SIZE,          # <<< Sử dụng ảnh đã resize
    use_augmentation=USE_AUGMENTATION,
    num_workers=NUM_WORKERS
)

if train_loader is None:
    print("Không thể tải dữ liệu. Kết thúc chương trình.")
    exit()

print("Khởi tạo ResNet18 pre-trained model")
weights = models.ResNet18_Weights.IMAGENET1K_V1
model = models.resnet18(weights=weights)

# --- Thay thế lớp phân loại cuối cùng (Fully Connected layer) ---
num_ftrs = model.fc.in_features # Lấy số lượng features đầu vào của lớp fc gốc
model.fc = nn.Linear(num_ftrs, num_classes) # Tạo lớp fc mới cho 100 lớp CIFAR-100
print(f"Đã thay thế lớp cuối cùng bằng lớp Linear({num_ftrs}, {num_classes})")

# (Tùy chọn) In tên các tham số để xem cấu trúc nếu cần
# for name, param in model.named_parameters():
#      print(name)

# (Tùy chọn) Đóng băng các lớp trừ lớp cuối - Chiến lược fine-tuning khác
# for name, param in model.named_parameters():
#     if "fc" not in name: # Chỉ đóng băng các lớp không phải lớp fc mới
#          param.requires_grad = False
# print("Đã đóng băng các lớp trừ lớp phân loại cuối.")
# # Khi đóng băng, cần đảm bảo optimizer chỉ tối ưu các tham số không bị đóng băng
# optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=LEARNING_RATE, momentum=0.9, weight_decay=5e-4)


# --- Huấn luyện ---
print("Bắt đầu fine-tuning ResNet18...")
train_model(model, train_loader, test_loader, NUM_EPOCHS, LEARNING_RATE, device, model_name="resnet18")